<a href="https://colab.research.google.com/github/ko-ko-go/Data-flow-Credit-Scoring/blob/main/creditscoring.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 1. ติดตั้ง Library ที่จำเป็น (เผื่อยังไม่มี)
!pip install kagglehub pandas-gbq db-sqlite3

import kagglehub
import sqlite3
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from google.colab import auth
import pandas_gbq
import os

# --- STEP 1: LOAD DATA ---
# Download latest version using kagglehub
path = kagglehub.dataset_download("urstrulyvikas/lending-club-loan-data-analysis")
print("Dataset downloaded to:", path)

# หาไฟล์ CSV ใน folder ที่โหลดมา (แก้ปัญหา Path ไม่ตรง)
csv_file = [f for f in os.listdir(path) if f.endswith('.csv')][0]
full_path = os.path.join(path, csv_file)

df = pd.read_csv(full_path) # อ่านจาก path จริง
print("Data Loaded Successfully!")

# --- STEP 2: PREPARE & MODELING (เหมือนเดิม) ---
# จำลอง Database (SQLite)
conn = sqlite3.connect(':memory:')
df.to_sql('loan_data', conn, index=False)

# SQL Query
sql_query = """
SELECT
    *,
    CASE
        WHEN fico >= 750 THEN 'Excellent'
        WHEN fico >= 700 THEN 'Good'
        WHEN fico >= 660 THEN 'Fair'
        ELSE 'Poor'
    END AS fico_category,
    EXP("log.annual.inc") AS annual_income
FROM loan_data
"""
df_processed = pd.read_sql(sql_query, conn)

# Modeling
df_model = pd.get_dummies(df_processed, columns=['purpose', 'fico_category'], drop_first=True)
X = df_model.drop(['not.fully.paid', 'annual_income', 'credit.policy'], axis=1)
y = df_model['not.fully.paid']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
model = LogisticRegression(max_iter=1000)
model.fit(X_train, y_train)

df_processed['pd'] = model.predict_proba(X)[:, 1]
df_processed['grade'] = pd.qcut(df_processed['pd'], q=[0, 0.2, 0.4, 0.6, 0.8, 1.0], labels=['A', 'B', 'C', 'D', 'F'])

# --- STEP 3: UPLOAD TO BIGQUERY ---
# ยืนยันตัวตน
auth.authenticate_user()

import pandas_gbq

# 1. ตั้งค่า Project ID ให้ถูกต้อง (อย่าลืมแก้ตรงนี้เป็น ID จริงของคุณนะครับ)
project_id = 'creditscoring-12345'  # <--- ใส่ Project ID ที่ถูกต้องตรงนี้
dataset_id = 'credit_data'
table_name = 'scoring_results'

# --- 2. แก้ไขชื่อคอลัมน์ (สำคัญ!) ---
# เปลี่ยนจุด (.) เป็นขีดล่าง (_) ให้หมด เพื่อให้ BigQuery รับได้
df_processed.columns = df_processed.columns.str.replace('.', '_')
print("✅ แก้ชื่อคอลัมน์เรียบร้อย (เช่น credit.policy -> credit_policy)")

# 3. ส่งข้อมูล
destination_table = f"{project_id}.{dataset_id}.{table_name}"
print(f"🚀 กำลังส่งข้อมูลไปยัง: {destination_table} ...")

df_processed.to_gbq(
    destination_table,
    project_id=project_id,
    if_exists='replace'
)

print("🎉 เรียบร้อย! ข้อมูลถูกส่งไป BigQuery แล้ว")

Using Colab cache for faster access to the 'lending-club-loan-data-analysis' dataset.
Dataset downloaded to: /kaggle/input/lending-club-loan-data-analysis
Data Loaded Successfully!


/usr/local/lib/python3.12/dist-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


✅ แก้ชื่อคอลัมน์เรียบร้อย (เช่น credit.policy -> credit_policy)
🚀 กำลังส่งข้อมูลไปยัง: creditscoring-12345.credit_data.scoring_results ...


/tmp/ipython-input-2892511073.py:78: FutureWarning: to_gbq is deprecated and will be removed in a future version. Please use pandas_gbq.to_gbq instead: https://pandas-gbq.readthedocs.io/en/latest/api.html#pandas_gbq.to_gbq
  df_processed.to_gbq(
100%|██████████| 1/1 [00:00<00:00, 8019.70it/s]

🎉 เรียบร้อย! ข้อมูลถูกส่งไป BigQuery แล้ว
